In [497]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from IPython.display import display

# OSFDM Function

In [499]:
def Thomas_Algorithm(a, b, c, d):

    n = len(d)
    vec=np.zeros(n)
    [aa,bb,cc,dd] = map(np.array, [a, b, c, d])

    for i in range(1,n):
        mult   = aa[i]/bb[i-1]
        bb[i] -= mult*cc[i-1]
        dd[i] -= mult*dd[i-1]

    vec[n-1] = dd[n-1]/bb[n-1]

    for i in range(n-2, -1, -1):
        vec[i] = (dd[i]-cc[i]*vec[i+1])/bb[i]

    return vec
    
def OSFDM (coef_list, u, parsing):

    coef = coef_list
    a_i = coef[0]
    b_i = coef[1]
    c_i = coef[2]
    a_j = coef[3]
    b_j = coef[4]
    c_j = coef[5]

    d_i = np.zeros(parsing-2)
    d_j = np.zeros(parsing-2)

    tmp = u
    # Solve with x-direction
    for j in range(1, parsing-1):
        d_i[0:parsing-1] = tmp[1:parsing-1, j] / k + (rho * x_vol * y_vol *  x[1:parsing-1] * y[j]) / 2 * \
                          (tmp[2:parsing, j+1] - tmp[2:parsing, j-1] - tmp[0:parsing-2, j+1] + tmp[0:parsing-2, j-1] ) / \
                          (4*hx ** 2)
        u[1:parsing-1, j] = Thomas_Algorithm(a_i, b_i, c_i, d_i)
    

    tmp = u
    # Solve with y-direction
    for i in range(1, parsing-1):
        d_j[0:parsing-1] = tmp[i, 1:parsing-1] / k + (rho * x_vol * y_vol * x[i] * y[1:parsing-1]) / 2 * \
                          (tmp[i+1, 2:parsing] - tmp[i+1,0:parsing-2] - tmp[i-1,2:parsing] + tmp[i-1,0:parsing-2])  / \
                          (4*hy**2)
                        
        u[i, 1:parsing-1] = Thomas_Algorithm(a_j, b_j, c_j, d_j)

    return u

def Two_Star_ELS_OSFDM (facevalue, x0_price, y0_price, parsing, rf, T, x_vol, y_vol, rho, x_q, y_q, barrier, coupon, strike):
    
    dim = [parsing, parsing]
    [u, w] = map(np.zeros, [dim, dim])

    for i in range(parsing):
        for j in range(parsing):

            if (x[i] < barrier * x0_price) or (y[j] < barrier*y0_price):

                if np.minimum(x[i], y[j]) == x[i]:
                    u[i,j] = np.minimum(x[i], y[j]) / x0_price * facevalue
                    w[i,j] = np.minimum(x[i], y[j]) / x0_price * facevalue

                elif y[j] < barrier*y0_price:
                    u[i,j] = np.minimum(x[i], y[j]) / y0_price * facevalue
                    w[i,j] = np.minimum(x[i], y[j]) / y0_price * facevalue



            elif (x[i] <= strike[0] * x0_price) or (y[j] <= strike[0] * y0_price):
                
                if x[i] <= strike[0] * x0_price:
                    u[i,j] = facevalue * (1 + coupon[0]) 
                    w[i,j] = np.minimum(x[i], y[j]) / x0_price * facevalue

                elif y[j] <= strike[0] * y0_price:
                    u[i,j] = facevalue * (1 + coupon[0]) 
                    w[i,j] = np.minimum(x[i], y[j]) / y0_price * facevalue

            else:
                u[i,j] = facevalue * (1 + coupon[0])
                w[i,j] = facevalue * (1 + coupon[0])

    [a_i, b_i, c_i, a_j, b_j, c_j] = map(np.zeros, [parsing-2, parsing-2, parsing-2, \
                                                parsing-2, parsing-2, parsing-2])

    a_i[:] = -0.5 * (x_vol * x[1:parsing-1]/hx) ** 2 + (rf - x_q)*x[1:parsing-1] / (2*hx)
    b_i[:] = 1/k  + (x_vol * x[1:parsing-1]/hx) ** 2 + (rf - x_q)*0.5
    c_i[:] = -0.5 * (x_vol * x[1:parsing-1]/hx) ** 2 - (rf - x_q)*x[1:parsing-1] / (2*hx)

    a_j[:] = -0.5 * (y_vol * y[1:parsing-1]/hy)**2 + (rf - x_q)*y[1:parsing-1] / (2*hy)
    b_j[:] = 1/k  + (y_vol * y[1:parsing-1]/hy)**2 + (rf - y_q) / 2
    c_j[:] = -0.5 * (y_vol * y[1:parsing-1]/hy)**2 - (rf - x_q)*y[1:parsing-1] / (2*hy)

    coef_list = [a_i, b_i, c_i, a_j, b_j, c_j]

    checker = 0
    for n in range(numT):
        
        if (n==step[checker]):
            x_Barrier_i = np.min(np.where(x >= x0_price * strike[checker+1]))
            y_Barrier_j = np.min(np.where(y >= y0_price * strike[checker+1]))
            
            u[x_Barrier_i:parsing-1, y_Barrier_j:parsing-1] = facevalue * (1 + coupon[checker+1])
            w[x_Barrier_i:parsing-1, y_Barrier_j:parsing-1] = facevalue * (1 + coupon[checker+1])

            checker += 1

        x_Barrier_i = np.min(np.where(x >= x0_price * barrier))
        y_Barrier_j = np.min(np.where(y >= y0_price * barrier))

        u[:, 0:y_Barrier_j+1] = w[:,0:y_Barrier_j+1]
        u[0:x_Barrier_i+1, :] = w[0:x_Barrier_i+1, :]
        
        u = OSFDM(coef_list, u, parsing)
        w = OSFDM(coef_list, w, parsing)


    x_idx = np.where((x0_price-1 <= x) & (x < x0_price+1))[0][0]
    y_idx = np.where((y0_price-1 <= y) & (y < y0_price+1))[0][0]

    return x_idx, y_idx, u

# Parameter

In [500]:
x_vol         = 0.3888      # Volatility for Starbucks
y_vol         = 0.3962      # Volatility for Netflix
rho           = 0.5287       # Correlation between Underlyings
x_q           = 0.013      # 180영업일의 평균 배당수익률 (SK하이닉스)
y_q           = 0.0016      # 180영업일의 평균 배당수익률 (NAVER)
T             = 3           # Maturity in Years
rf            = 0.0282   # Risk-Free Rates= 3M LIBOR as of Oct 22 2020
barrier       = 0.5        # Knock-In Barrier
coupon        = np.array([0.051, 0.102, 0.153, 0.204, 0.255, 0.306])[::-1]  # Coupons
strike        = np.array([0.85, 0.85, 0.80, 0.80, 0.75, 0.75])[::-1]  # Strike Prices

facevalue     = 10000
x0_price      = 100
y0_price      = 100
x0_max        = x0_price * 3                        
y0_max        = y0_price * 3  
parsing       = 91         # Number of Intervals x y
hx            = x0_max / parsing                 # Intervals for Underlyings
hy            = y0_max / parsing                

x             = np.linspace(0, x0_max, parsing)  # The First Underlying
y             = np.linspace(0, y0_max, parsing) 

numT          = 360 * T                            # Number of Time Steps
k             = T / numT
step = np.int0((np.linspace(0, T, 2 * T +1) * numT/3))[1:]

In [501]:
x_idx, y_idx, u = Two_Star_ELS_OSFDM(facevalue, x0_price, y0_price, parsing, rf, T, x_vol, y_vol, rho, x_q, y_q, barrier, coupon, strike)
print(f'Two Star ELS PRICE by OSFDM = {np.round(u[x_idx, y_idx].item(),6)}')

Two Star ELS PRICE by OSFDM = 8897.683918


# Greeks (OSFDM)

In [502]:
dsig_x = 0.01 * x_vol
dsig_y = 0.01 * y_vol
dsig = (dsig_x + dsig_y)/2
x2_idx, y2_idx, u2 = Two_Star_ELS_OSFDM(facevalue, x0_price, y0_price, parsing, rf, T, x_vol+dsig_x, y_vol+dsig_y, rho, x_q, y_q, barrier, coupon, strike)

In [503]:
v1 = u[x_idx+1, y_idx+1] 
v2 = u[x_idx, y_idx]
v3 = u[x_idx-1, y_idx-1]

h = hx * 0.5 + hy * 0.5

Delta = (v1 - v3)/h * 0.01
Delta_x = (v1)
Gamma = (v1 - 2*v2 + v3)/(h**2)


# x2_idx, y2_idx, u2 = Two_Star_ELS_OSFDM(facevalue, x0_price, y0_price, parsing, rf, T, x_vol+dsig_x, y_vol+dsig_y, rho, x_q, y_q, barrier, coupon, strike)

Vega = (v2-u2[x2_idx, y2_idx])*0.01**2/dsig

print(f'Two Star ELS "DELTA" by OSFDM = {Delta}')
print(f'Two Star ELS "GAMMA" by OSFDM = {Gamma}')
print(f'Two Star ELS "Vega" by OSFDM = {Vega}')




Two Star ELS "DELTA" by OSFDM = 1.2527498422259764
Two Star ELS "GAMMA" by OSFDM = -1.985277714180559
Two Star ELS "Vega" by OSFDM = 1.066787456716498


# SIMULATION METHOD

In [504]:
'''---------------= INPUT DATA ----------------'''

'''----- 배당은 각각 한국전력 배당수익률 5년 평균치 이용, 한국전력 = 5.27% / 하나금융지주 = 4.82% ----'''

num = 10000
num_asset = 2
step = 3 * 90
T = 3
dt = T / step #/num_time_step
r = 0.0282 
q1 = 0.013 
q2 = 0.0016
rho = 0.5287
sigma1 = 0.3888 
sigma2 = 0.3962 
rate = 0.102

s1_0 = 10000
s2_0 = 10000

c_tag = [0.85, 0.85, 0.80, 0.80, 0.75, 0.75]

b = 0.5

case_dict = {
    1 : s1_0 * (1 + 0.5 * rate) / (1 + 0.5 * r),
    2 : s1_0 * (1 + 1.0 * rate) / (1 + 1.0 * r),
    3 : s1_0 * (1 + 1.5 * rate) / (1 + 1.5 * r),
    4 : s1_0 * (1 + 2.0 * rate) / (1 + 2.0 * r),
    5 : s1_0 * (1 + 2.5 * rate) / (1 + 2.5 * r),
    6 : s1_0 * (1 + 3.0 * rate) / (1 + 3.0 * r),
    7 : s1_0 / (1 + 3.0 * r),
}

case_tag = (2 * step/(len(case_dict)-1) * np.array([0.5, 1, 1.5, 2, 2.5, 3])).astype(int)


'''---------------- Equations and Simulation -----------------'''

path1 = np.ones((num, 1)) * s1_0
path2 = np.ones((num, 1)) * s2_0

z1 = np.random.normal(size=(step, num)).T
z2 = rho * z1 + np.sqrt(1-rho**2) * np.random.normal(size=(step, num)).T

x1 = np.exp( (r - q1 - 0.5 * (sigma1 **2)) * dt + sigma1 * np.sqrt(dt) * z1)
x2 = np.exp( (r - q2 - 0.5 * (sigma2 **2)) * dt + sigma2 * np.sqrt(dt) * z2)

for i in range(step):
    next_path = x1[:,i].reshape(num, 1) * path1[:, -1].reshape(num, 1)
    path1 = np.c_[path1, next_path]

for i in range(step):
    next_path = x2[:,i].reshape(num, 1) * path2[:, -1].reshape(num, 1)
    path2 = np.c_[path2, next_path]

path1_df = pd.DataFrame(data=path1, index=[x for x in range(num)], columns=[x for x in range(step+1)])
path2_df = pd.DataFrame(data=path2, index=[x for x in range(num)], columns=[x for x in range(step+1)])

check1 = pd.DataFrame(data=path1, index=[[x for x in range(num)],[0 for x in range(num)]], columns=[x for x in range(step+1)])
check2 = pd.DataFrame(data=path2, index=[[x for x in range(num)],[1 for x in range(num)]], columns=[x for x in range(step+1)])


df = pd.concat([check1, check2]).sort_index()
display(df)


'''----------------------- Case by Case --------------------------'''

'''Case1 Check'''
'''모두 6개월 시점에서 85 이상일때'''
case1_arr = ((path1_df[case_tag[0]] >= (s1_0 * c_tag[0]) * 1) * (path2_df[case_tag[0]] >= (s2_0 * c_tag[0]) * 1)) * 1

'''Case2 Check'''
'''모두 12개월 시점에서 85 이상일때'''
case2_arr = (((path1_df[case_tag[1]] >= (s1_0 * c_tag[1])) * 1) * ((path2_df[case_tag[1]] >= (s2_0 * c_tag[1])) * 1))
case2_arr = np.where((case2_arr - case1_arr) > 0, 1, 0)

'''Case3 Check'''
'''모두 18개월 시점에서 80 이상일때'''
case3_arr = (((path1_df[case_tag[2]] >= (s1_0 * c_tag[2])) * 1) * ((path2_df[case_tag[2]] >= (s2_0 * c_tag[2])) * 1))
case3_arr = np.where((case3_arr - case2_arr - case1_arr) > 0, 1, 0)

'''Case4 Check'''
'''모두 24개월 시점에서 80 이상일때'''
case4_arr = (((path1_df[case_tag[3]] >= (s1_0 * c_tag[3])) * 1) * ((path2_df[case_tag[3]] >= (s2_0 * c_tag[3])) * 1))
case4_arr = np.where((case4_arr - case3_arr - case2_arr - case1_arr) > 0, 1 ,0)

'''Case5 Check'''
'''모두 30개월 시점에서 75 이상일때'''
case5_arr = (((path1_df[case_tag[4]] >= (s1_0 * c_tag[4])) * 1) * ((path2_df[case_tag[4]] >= (s2_0 * c_tag[4])) * 1))
case5_arr = np.where((case5_arr - case4_arr - case3_arr - case2_arr - case1_arr) > 0, 1 ,0)

'''Case6 Check'''
'''36개월 시점에서 75 이상일때'''
case6_arr = ( (((path1_df >= (s1_0 * c_tag[5])).prod(axis=1) * (path2_df >= (s2_0 * c_tag[5])).prod(axis=1))) * (( (((path1_df[case_tag[5]] >= (s1_0 * b)) * 1) * ((path2_df[case_tag[5]] >= (s2_0 * b)) * 1)) - 1) ** 2) )
case6_arr = np.where((case6_arr - case5_arr - case4_arr - case3_arr - case2_arr - case1_arr) > 0, 1, 0)

'''Case7 Check, 조기상환 경우는 제외함'''
'''둘다 한번이라도 60 미만으로 하락한 적이 있고, 만기 평가 가격이 한 종목 이라도 70 미만인 경우'''
case7_arr = ( (((path1_df >= (s1_0*b)).prod(axis=1) * (path2_df >= (s2_0*b)).prod(axis=1))) )
case7_arr = np.where((case7_arr - case6_arr - case5_arr - case4_arr - case3_arr - case2_arr - case1_arr) > 0, 1, 0)

case8_arr = np.ones((num)) - (case1_arr + case2_arr + case3_arr + case4_arr + case5_arr + case6_arr + case7_arr)

case = case1_arr * 1 + case2_arr * 2 + case3_arr * 3 + case4_arr * 4 + case5_arr * 5 + case6_arr * 6 + case7_arr * 7 + case8_arr * 8

'''--------------------- Values from case by case -------------------------'''

path1_df['case'] = case.astype(int)
path2_df['case'] = case.astype(int)


y_arr = np.where(path1_df['case'] == 1, case_dict[1],\
        np.where(path1_df['case'] == 2, case_dict[2],\
        np.where(path1_df['case'] == 3, case_dict[3],\
        np.where(path1_df['case'] == 4, case_dict[4],\
        np.where(path1_df['case'] == 5, case_dict[5],\
        np.where(path1_df['case'] == 6, case_dict[6],\
        np.where(path1_df['case'] == 7, case_dict[6],
        np.where(path1_df['case'] == 8, \
        0, 0))))))))

y_arr = y_arr + np.c_[((path1_df['case'] == 8) * path1_df[step])/s1_0, ((path1_df['case'] == 8) * path2_df[step])/s2_0].min(axis=1) * case_dict[7]

path1_df['Y'] = y_arr
path2_df['Y'] = y_arr

display(path1_df[['case', 'Y']])


'''------------------------- Price and Standard, and DF sorting -------------------------'''

value = path2_df['Y'].mean()
sample_std = np.sqrt(sum((path2_df['Y'] - value)**2)/(num-1))
mc_simul_error = sample_std/ np.sqrt(num)

prob_df = pd.DataFrame(data=path1_df.groupby('case').size().index, columns=['case']).set_index('case')
prob_df['count'] = path1_df.groupby('case').size()
prob_df['prob(%)'] = (path1_df.groupby('case').size() / num) * 100

display(prob_df)

print(f'ELS price = {value}, standard error = {mc_simul_error}')

0             1             2             3             4    \
0    0  10000.0  10408.584628  10715.162789  10531.810688  10814.684804   
     1  10000.0  10040.028918  10617.641592  10792.616392  11085.612053   
1    0  10000.0  10590.504684  10215.348475   9841.227090  10152.456188   
     1  10000.0  10494.359080  10031.921923   9282.656896   8868.192023   
2    0  10000.0   9542.486272   9118.531342   9686.499081   9626.326978   
...         ...           ...           ...           ...           ...   
9997 1  10000.0   9953.701580   9389.128577   9119.283930   9005.803086   
9998 0  10000.0   9418.251079   9150.717527   9458.116443   9553.299358   
     1  10000.0   9300.111558   8965.578826   9177.994640   9316.372896   
9999 0  10000.0  10030.883340  10118.844342  10067.423305   9811.740329   
     1  10000.0   9814.531119   9252.104849   9371.418632   8569.838494   

                 5             6             7             8             9    \
0    0  11075.791369  11320.207015  10901.811399  10219.077016   9795.476757   
     1  11176.182363  11815.740576  11773.413434  11083.845119  11202.740770   
1    0  10116.123772  10879.684064  11305.967212  11585.212626  11672.600746   
     1   8555.775926   9178.383312   9345.470022   9375.579032   9292.917254   
2    0  10216.677682  10997.965797  11258.918899  11703.486043  12512.319439   
...              ...           ...           ...           ...           ...   
9997 1   8605.431066   8893.569491   8627.739381   8985.889010   9342.475712   
9998 0   9452.487454   9313.024312   8384.152670   8751.307075   8292.132085   
     1   8769.335638   7880.804608   7298.701850   7306.895118   7381.466669   
9999 0   9588.561966   9626.417326   9833.686255   9479.164298   9684.555278   
     1   8075.364849   8156.037387   8415.666684   7864.945477   7790.275985   

        ...           261           262           263           264  \
0    0  ...  14141.352388  13669.225975  12706.304129  13226.264437   
     1  ...  10735.740171  10109.129005   9715.550460   9615.042152   
1    0  ...  18806.509112  18196.378355  18305.214343  17894.691733   
     1  ...  19831.324117  19153.903570  20236.676203  20249.912691   
2    0  ...  17669.335976  18462.426216  19173.171246  19607.954094   
...     ...           ...           ...           ...           ...   
9997 1  ...   4706.957634   4813.501461   5025.013709   5096.997856   
9998 0  ...   7795.013183   7756.907759   7943.919132   8504.523007   
     1  ...  11372.723823  10593.197933  10613.608221  10962.512112   
9999 0  ...  12692.917222  12580.895409  13061.721015  13528.200005   
     1  ...   4074.707712   4266.788648   4347.048205   4605.627498   

                 265           266           267           268           269  \
0    0  13514.878021  13312.700720  13601.184592  13837.534202  13507.367063   
     1   9482.018757   9420.990380   9556.142944   9754.506728   9877.307132   
1    0  18078.880073  17356.458708  16582.680419  16204.916490  15486.270699   
     1  20532.521264  19178.223188  18407.066212  18238.229298  18064.327696   
2    0  18557.927381  18029.182543  17234.587424  17147.358663  17331.057450   
...              ...           ...           ...           ...           ...   
9997 1   5098.498369   5210.055390   5208.600917   5006.271981   5292.647515   
9998 0   8296.745677   7972.157433   7910.244913   7658.665767   7713.844904   
     1  10607.716713  10309.790184  10862.588905  10767.534717  10987.863256   
9999 0  13145.629666  11710.766993  12030.831434  11549.469229  10964.358708   
     1   4332.963728   4179.112682   4290.243093   4165.955897   4120.650700   

                 270  
0    0  14188.619067  
     1  10587.505571  
1    0  16131.865441  
     1  18766.159885  
2    0  17438.562083  
...              ...  
9997 1   5283.923481  
9998 0   7780.795224  
     1  10904.931115  
9999 0  11720.813512  
     1   4342.071941  

[20000 rows x 271 columns]

,case,Y
0,1,10363.869441
1,3,11062.074259
2,1,10363.869441
3,1,10363.869441
4,1,10363.869441
...,...,...
9995,2,10717.759191
9996,8,4472.752188
9997,8,4871.771603
9998,2,10717.759191


,count,prob(%)
case,,
1,5426,54.26
2,906,9.06
3,626,6.26
4,300,3.00
5,224,2.24
7,83,0.83
8,2435,24.35


ELS price = 8911.472276541374, standard error = 30.442545280319916
